## Health Data Reusability Project

This notebook is an informal investigation into the technologies needed to take the data contained in "open data" publications from the UK Department of Health. This will allow researchers to automate computations and respond more speedily to changes.

Some of this is ugly, some of it will doubtless be unnecessary, but it shows at least some of the preliminary work that goes into getting one's thinking straightened out about a particular program or set of programs.

In [ ]:
import openpyxl as xl

Note that this softare cannot read ".xls" files. `wb = xl.load_workbook("data/gpearnextime.xls")` raises an exception, so I toook the quick route and converted it to a ".xlsx" file with Word before further processing.

It might be worth investigating the older `xlrd` module, which can reals ".xls" files (though sadly there appears
to be no easy way to write them out as ".xslx" files which I had hopes `xlwt` might have provided. I suspect that there will be an easy fix for this, but I'll need to speak to Chris Withers.

In [ ]:
wb = xl.load_workbook("data/gpearnextime.xlsx")

In [ ]:
wb.sheetnames

In [ ]:
ws = wb.get_sheet_by_name('1a. GPMS Cash Terms ')

In [ ]:
ws["B7"].value

In [ ]:
for i in range(1, 200):
    print(ws["B{}".format(i)].value, ws["C{}".format(i)].value)

Note that the date values and the footnote numbers run together to give a single string value.
That means some parsing has to be applied to separate it into a `(date, footnote)` pair, whose
second member will be `None` if no notes apply.
From an openness point of view it would be much better to have a separate column for the footnotes that should be applied to the row.
Then again, from an openness point of view it would be better not to use Excel spreadsheets 

Turns out that may not be as useful as I thought. It would probably be eaiser to maintain the column values as part ofthe processing logic.

(This was borne out when I wrote a non-terminting loop when experimenting with the code below)

In [ ]:
ws["B3"].value

In [ ]:
def year_refs(s):
    """Separate the year string into the year plus the list of references"""
    return s[:7], s[7:].split(",")

In [ ]:
ws["d81"].value

In [ ]:
def num_val(val):
    return 0 if val == "-" else val

In [ ]:
num_val(32.456)

In [ ]:
num_val("-")

In [ ]:
3 == "banana"

In [ ]:
cell = ws["B3"]

Probably a good idea to look at how we can find the relevant areas in a worksheet, then analyze the content of those areas (which will vary in size, increasing as the years go by.

In [ ]:
ws["B3"].value # Sheet heading

In [ ]:
ws["B5"].value # Table heading

In [ ]:
cells = ws.get_cell_collection()

In [ ]:
from collections import defaultdict

cols_in_row = defaultdict(list)

for cell in cells:
    if cell.value is not None:
        cols_in_row[cell.row].append(cell.column)

In [ ]:
max_row = max(c for c in sorted(cols_in_row.keys()))
max_row

Note that cell J43 has a spurious value that should really be ignored. Wonder how long that's been there and who knows it is ...

In [ ]:
cols_in_row[43].remove('J')
cols_in_row[43]

In [ ]:
pixels = [] # straight list of pixel values for graphic
matrix = []
#print("  ".join(list("ABCDEFG"))) # Column headings
for row_num in range(max_row):
    cols = cols_in_row[row_num]
    row_string = []
    row_matrix = []
    for col_name in "ABCDEFG":
        row_string.append("*" if col_name in cols else " ")
        row_matrix.append(col_name in cols)
    #print("  ".join(row_string))
    matrix.append(row_matrix)
    pixels += [1-p for p in row_matrix] + [1]*7 # add pixel row plus blank row

In [ ]:
from PIL import Image
im = Image.new("1", (7, 198*2))

In [ ]:
im.putdata(pixels)

In [ ]:
im.resize((7*20, 198*4))

This visualization makes the pattern of the tables more obvious.
Each table begubs with a row with a single cell, followed by two cells with six rows and a number of rows with five cells.
Now, it would be possible to construct a vector with the number of cells in each row, and then search
that for patterns characteristic of the start of a table.
Whenever you find yourself thinking "pattern," though, it's worth considering using Python's `re`
regular expression-based pattern-matching algorithm.
Since no row has more than seven cells we can construct a ___string of row lengths___ and then
use pattern matching to find the starting positions of the tables.
The task then simplifies to finding the string `"1665"`.

In [ ]:
str_sizes = "".join(str(sum(x for x in row)) for row in matrix)

In [ ]:
import re
for m in re.finditer("(?P<m>1665)", str_sizes):
    print(m.span()[0])

Maybe there's some easier way to determing the shape without all these complex manipulations.
I don't know about you, but I often find my second approach to a problem is more intelligent
than the first (that's why we are often recommended to write a prototype _and then throw it away_).
So consider everything above as prototypical, offering insight into the necessary analysis but using a horribly inefficient algorithm.

In [ ]:
len(ws.columns)

In [ ]:
col_counts =[sum(cell.value is not None for cell in column) for column in ws.columns]

In [ ]:
valid_cols = [i for (i, ct) in  enumerate(col_counts) if ct > 1]
valid_cols

In [ ]:
row_counts =[sum(cell.value is not None for cell in row) for row in ws.rows]
valid_rows = [i for (i, ct) in  enumerate(row_counts) if ct > 0]
len(valid_rows)

In [ ]:
ncols = len(valid_cols)
max_row_num = max(valid_rows)
pixels = []
row_counts = []
for row in range(max_row_num):
    cell_strings = []
    row_pixels = []
    for col in valid_cols:
        value = ws.rows[row][col].value
        row_pixels.append(value is None)
    pixels += row_pixels*3 + [1]*ncols # blank line
    row_counts.append(sum(1-pixel for pixel in row_pixels))

In [ ]:
im = Image.new("1", (ncols, 4*(max_row_num)))
im.putdata(pixels)
im.resize((14*ncols, 8*(max_row_num+1)))

It appears that this techmique can be fairly effectively used to get an idea of the shape of a worksheet.
The next step will be to turn that into a function with the worksheet as a parameter, and apply it to all the sheets on a workbook.

In [ ]:
%matplotlib inline

In [ ]:
def visualize(ws):
    col_counts =[sum(cell.value is not None for cell in column) for column in ws.columns]
    valid_cols = [i for (i, ct) in  enumerate(col_counts) if ct > 1]
    row_counts =[sum(cell.value is not None for cell in row) for row in ws.rows]
    valid_rows = [i for (i, ct) in  enumerate(row_counts) if ct > 0]
    ncols = len(valid_cols)
    max_row_num = max(valid_rows)
    pixels = []
    row_counts = []
    for row in range(max_row_num):
        cell_strings = []
        row_pixels = []
        for col in valid_cols:
            value = ws.rows[row][col].value
            row_pixels.append(value is None)
        pixels += row_pixels*3 + [1]*ncols # blank line
        row_counts.append(sum(1-pixel for pixel in row_pixels))
    im = Image.new("1", (ncols, 4*(max_row_num)))
    im.putdata(pixels)
    return im.resize((14*ncols, 8*(max_row_num+1))), row_counts

In [ ]:
images = []; row_counts = []
for ws in wb.worksheets:
    if ws.sheet_state != "hidden": # exclude hidden sheets
        image, counts = visualize(ws)
        images.append(image)
        row_counts.append(counts)

In [ ]:
LEFT_MARGIN = 4
im_width = sum(i.size[0] for i in images)+(len(images)-1)*LEFT_MARGIN
im_height = max(i.size[1] for i in images)
im_width, im_height

In [ ]:
len(row_counts)

In [ ]:
x_offset = 0
big_image = Image.new("1", (im_width, im_height), 1)
for i, im in enumerate(images):
    big_image.paste(im, (x_offset, 0))
    x_offset += im.size[0]+LEFT_MARGIN

In [ ]:
big_image

In [ ]:
ws = wb.worksheets[2]
ws

In [ ]:
# Page header is in B3 always?
page_header = ws["B3"].value
print(page_header)

In [ ]:
counts_string = "".join(str(n) for n in row_counts[2])
counts_string

Looking for the pattern `"1665"`\* in `counts_string` finds a six-column table.
The first line is the name of the table.
The second line is the column headings.
The remainder of the table is a number of repeating groups.
The first column is special because unchanged values aren't repeated (which is why subsequent lines only have five elements).
While this is helpful for the human reader's comprehension it has to be corrected for the computer.

\* Yes, this is a fix - that string was chosen because I knew there were six columns in the tables.
We may or may not get to the computation of the number of columns later.

In [ ]:
table_starts = [x.start("X") for x in re.finditer("(?P<X>1665)", counts_string)]
table_starts

In [ ]:
table_lens = [x.end("X")-x.start("X")-1 for x in re.finditer("(1(?P<X>665([56]+)))", counts_string)]
table_lens

In [ ]:
number_of_tables = len(table_starts)
number_of_tables

Let's do a little work on the first table.
Also, let's learn how to access the elements we need in order to construct a usable data source.
The crucial facts for each table are the number of groups, the number of rows in each group
and the number of columns in the table.
When you think about it this is simply a description of a three-dimensional structure.
The first table reports figures over the whole UK.
The remainder analyze that information geographically, adding a fourth dimension to the data.

Let's do a little work on the first table to learn what we'll need to do in the general case.

Firstly, let's see how to access the various "chunks" of the table, beginning with its title.

In [ ]:
start_row = table_starts[0]
table_len = table_lens[0]
ws["B5"].value, ws["B5"], ws.columns[1][4], ws.columns[1][start_row]

In [ ]:
title_cell = ws.columns[1][start_row]
title_cell.value

Next we need to extract the column names, which are on the row following the title.

In [ ]:
table_cols = 6
# was int(counts_string[start_row+1]), but that depended on arcane knowledge in the building of the pattern
[c.value for c in ws.rows[start_row+1][1:table_cols+1]] # really should have computed that "6" from the pattern ...

In [ ]:
# This is a cheesy way to work out how many groups there are
pat = "(?P<X>(65+)+)"
m = re.search(pat, counts_string, start_row+2)
assert m.groups(0)[0].replace(m.groups(0)[1], "") == "" # only true for fixed groups

In [ ]:
group_count = len(m.groups(0)[0])//len(m.groups(0)[1])
group_count

In [ ]:
group_len = table_len//group_count
group_len

In [ ]:
data_start_row = start_row+2
group_cells = [[ws.rows[row][col].value for col in range(1, table_cols+1)]
                   for row in range(data_start_row, data_start_row+group_len)]

In [ ]:
group_cells

In [ ]:
import pandas as pd
group = pd.DataFrame(group_cells)

In [ ]:
group